In [2]:
import requests
import os

In [3]:
import pandas as pd

In [4]:
import pprint

In [5]:
import json

In [32]:
foursquare_api_key = os.environ['FOURSQUARE_STATS_API_KEY']

In [1]:
#%store -r lat_lon

In [42]:
lat_lon_init = pd.read_csv('../data/lat_lon.csv')
lat_lon = lat_lon_init.drop(columns=lat_lon_init.columns[0], axis=1)
lat_lon = lat_lon.values.tolist()


In [43]:
type(lat_lon)

list

In [44]:
lat_lon

[[53.35023, -6.279696],
 [53.357841, -6.251557],
 [53.356307, -6.273717],
 [53.349562, -6.278198],
 [53.336021, -6.26298],
 [53.359405, -6.276142],
 [53.33796, -6.24153],
 [53.343368, -6.27012],
 [53.334123, -6.265436],
 [53.344304, -6.250427],
 [53.338755, -6.262003],
 [53.347777, -6.244239],
 [53.336074, -6.252825],
 [53.330091, -6.268044],
 [53.350929, -6.265125],
 [53.341515, -6.256853],
 [53.348279, -6.254662],
 [53.35893, -6.280337],
 [53.344115, -6.237153],
 [53.343893, -6.280531],
 [53.347477, -6.28525],
 [53.339005, -6.300217],
 [53.344153, -6.233451],
 [53.334295, -6.258503],
 [53.340803, -6.267732],
 [53.351464, -6.255265],
 [53.333653, -6.248345],
 [53.343105, -6.277167],
 [53.341428, -6.24672],
 [53.338614, -6.248606],
 [53.341833, -6.231291],
 [53.346637, -6.246154],
 [53.343034, -6.263578],
 [53.346867, -6.230852],
 [53.3547, -6.272314],
 [53.33403, -6.260714],
 [53.346026, -6.243576],
 [53.330662, -6.260177],
 [53.342113, -6.310015],
 [53.357043, -6.263232],
 [53.348875

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [45]:
def foursquare_request(latitude, longitude):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        # retrieve latitude and longitude from list of bike station location coordinates, and convert to string
        "ll": f"{latitude},{longitude}",
        "radius": 1000,
        "categories": "13000, 19028, 19043", # search for dining and drinking, bus stops, and tourist information and service
        "sort": "DISTANCE",
        "fields": "name,distance,popularity,rating,categories" 
    }

    headers = {
        "Accept": "application/json",
        "Authorization": f"{foursquare_api_key}"
    }

    foursquare_result = requests.request("GET", url, params=params, headers=headers)

    return foursquare_result.json()

# create empty list to contain a dataframe for each location's associated information
final_location_data_list = []

# iterate through bike station locations
for location in lat_lon:
    latitude = location[0]
    longitude = location[1]
    print(f'getting results for {latitude}, {longitude}')
    result = foursquare_request(latitude, longitude) # assign location json results to a variable
    data = result.get('results') # extract data located in the 'results' header of the json file
    foursquare_df = pd.DataFrame(data) # convert location results to a pandas dataframe
    foursquare_df['latitude'] = latitude
    foursquare_df['longitude'] = longitude 
    foursquare_df['category'] = foursquare_df['categories'].map(lambda x: x[0].get('name')) # extract only the first category name from a venue's category info
    foursquare_df = foursquare_df.drop('categories', axis = 1) # remove the 'categories' column which contains a list of dictionaries
    final_location_data_list.append(foursquare_df)

final_df = pd.concat(final_location_data_list, axis=0) # convert list of location dataframes to one big dataframe

final_df.head(10)

getting results for 53.35023, -6.279696
getting results for 53.357841, -6.251557
getting results for 53.356307, -6.273717
getting results for 53.349562, -6.278198
getting results for 53.336021, -6.26298
getting results for 53.359405, -6.276142
getting results for 53.33796, -6.24153
getting results for 53.343368, -6.27012
getting results for 53.334123, -6.265436
getting results for 53.344304, -6.250427
getting results for 53.338755, -6.262003
getting results for 53.347777, -6.244239
getting results for 53.336074, -6.252825
getting results for 53.330091, -6.268044
getting results for 53.350929, -6.265125
getting results for 53.341515, -6.256853
getting results for 53.348279, -6.254662
getting results for 53.35893, -6.280337
getting results for 53.344115, -6.237153
getting results for 53.343893, -6.280531
getting results for 53.347477, -6.28525
getting results for 53.339005, -6.300217
getting results for 53.344153, -6.233451
getting results for 53.334295, -6.258503
getting results for 53.

,distance,name,popularity,rating,latitude,longitude,category
0,37,V-Face,0.963409,NaN,53.35023,-6.279696,Vegan and Vegetarian Restaurant
1,177,Light House Cinema,0.988546,9.1,53.35023,-6.279696,Indie Movie Theater
2,250,Nutbutter,0.957725,NaN,53.35023,-6.279696,Poke Restaurant
3,285,JJ's Bar,0.995858,8.3,53.35023,-6.279696,Cocktail Bar
4,286,Token,0.974035,8.7,53.35023,-6.279696,Arcade
5,287,The Reserve Bar,0.989603,7.5,53.35023,-6.279696,Whisky Bar
6,291,Old Jameson Distillery,0.998886,8.9,53.35023,-6.279696,Tour Provider
7,297,Proper Order Coffee Co.,0.984347,8.0,53.35023,-6.279696,Café
8,331,Bonobo,0.993887,8.5,53.35023,-6.279696,Bar
9,362,Wuff,0.967094,8.7,53.35023,-6.279696,Bistro


In [46]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 0 to 9
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   distance    1140 non-null   int64  
 1   name        1140 non-null   object 
 2   popularity  1120 non-null   float64
 3   rating      768 non-null    float64
 4   latitude    1140 non-null   float64
 5   longitude   1140 non-null   float64
 6   category    1140 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 71.2+ KB


In [61]:
final_df.to_csv('../data/foursquare_results.csv')

### Troubleshooting code snippets

run the foursquare request once, figure out where the category information is within the foursquare result structure.

In [58]:
url = "https://api.foursquare.com/v3/places/search"
params = {
    # retrieve latitude and longitude from list of bike station location coordinates, and convert to string
    "ll": "53.35023,-6.279696",
    "radius": 1000,
    "categories": "13000, 19028, 19043", # search for dining and drinking, bus stops, and tourist information and service
    "sort": "DISTANCE",
    "fields": "name,distance,popularity,rating,categories" 
}

headers = {
    "Accept": "application/json",
    "Authorization": f"{foursquare_api_key}"
}

foursquare_result = requests.request("GET", url, params=params, headers=headers)

pprint.pprint(foursquare_result.json())

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 53.35023,
                                                  'longitude': -6.279696},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
                                       'suffix': '.png'},
                              'id': 13377,
                              'name': 'Vegan and Vegetarian Restaurant'}],
              'distance': 37,
              'name': 'V-Face',
              'popularity': 0.9622954097517781},
             {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
                                       'suffix': '.png'},
                              'id': 13018,
                              'name': 'Pub'}],
              'distance': 131,
              'name': 'The Cobblestone',
              'popularity': 0.9895169813476534,
              'rating': 8.

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Points of interest:
- dining and drinking (13000)
    - coffee shops (13032)
    - brewery (13029)
    - bars (13003)
    - restaurant (13065)
- bus stop (19043)
- tourist information and service (19028)

POIs that were considered but not included:
- public bathroom (12092)
- historic and protected site (16020)
- souvenir store (17116)
- gift shops (17089)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Yelp categories of interest:
- food
- visitorcenters
- busstations

In [47]:
yelp_api_key = os.environ['YELP_API_KEY']

In [60]:
def yelp_request(latitude, longitude):
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": 1000,
        "categories": "bars,restaurants,coffeeshops,busstations,visitorcenters", # search for dining and drinking, bus stops, and tourist information and service
        "sort_by": "distance"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": f"BEARER {yelp_api_key}"
    }

    yelp_result = requests.request("GET", url, params=params, headers=headers)

    return yelp_result.json()

# create empty list to contain a dataframe for each location's associated information
final_yelp_location_data_list = []

# iterate through bike station locations
for location in lat_lon:
    latitude = location[0]
    longitude = location[1]
    print(f'getting results for {latitude}, {longitude}')
    result = yelp_request(latitude, longitude) # assign location json results to a variable
    data = result.get('businesses') # extract data located in the 'businesses' header of the json file
    yelp_df = pd.DataFrame(data) # convert location results to a pandas dataframe
    yelp_df['latitude'] = latitude
    yelp_df['longitude'] = longitude 
    try:
        yelp_df['category'] = yelp_df['categories'].map(lambda x: x[0].get('title')) # extract only the first category name from a venue's category info
    except IndexError as e: # skip coordinates that cause an IndexError
        print(f"{e}")
    #yelp_df = yelp_df.drop('categories', axis = 1) # remove the 'categories' column which contains a list of dictionaries
    yelp_df_1 = yelp_df.filter(['distance','name','review_count','rating','latitude','longitude','category'], axis=1) # return only columns of interest
    final_yelp_location_data_list.append(yelp_df_1) # add each location dataframe to master list

final_yelp_df = pd.concat(final_yelp_location_data_list, axis=0) # convert list of location dataframes to one big dataframe

final_yelp_df.head(50)

getting results for 53.35023, -6.279696
getting results for 53.357841, -6.251557
getting results for 53.356307, -6.273717
getting results for 53.349562, -6.278198
getting results for 53.336021, -6.26298
getting results for 53.359405, -6.276142
getting results for 53.33796, -6.24153
getting results for 53.343368, -6.27012
getting results for 53.334123, -6.265436
getting results for 53.344304, -6.250427
getting results for 53.338755, -6.262003
getting results for 53.347777, -6.244239
getting results for 53.336074, -6.252825
getting results for 53.330091, -6.268044
getting results for 53.350929, -6.265125
getting results for 53.341515, -6.256853
getting results for 53.348279, -6.254662
getting results for 53.35893, -6.280337
getting results for 53.344115, -6.237153
getting results for 53.343893, -6.280531
getting results for 53.347477, -6.28525
getting results for 53.339005, -6.300217
getting results for 53.344153, -6.233451
getting results for 53.334295, -6.258503
getting results for 53.

,distance,name,review_count,rating,latitude,longitude,category
0,45.574658,Lavanda,5,4.5,53.350230,-6.279696,Modern European
1,64.668846,Namaste India,27,3.5,53.350230,-6.279696,Indian
2,92.167605,Delaneys,6,3.5,53.350230,-6.279696,Pubs
3,119.866129,The Cobblestone,122,4.5,53.350230,-6.279696,Pubs
4,133.331760,Walshs,17,4.5,53.350230,-6.279696,Pubs
5,136.266132,Stir Cafe,1,5.0,53.350230,-6.279696,Irish
6,142.018024,Gutterball,4,2.0,53.350230,-6.279696,American (New)
7,157.409129,The Belfry,1,4.0,53.350230,-6.279696,Pubs
8,165.311008,Social Fabric Cafe,2,5.0,53.350230,-6.279696,Breakfast & Brunch
9,166.703864,Thundercut Alley,13,3.5,53.350230,-6.279696,Cocktail Bars


In [62]:
final_yelp_df.to_csv('../data/yelp_results.csv')

In [55]:
# url = "https://api.yelp.com/v3/businesses/search"
# params = {
#     "latitude": 53.333653,
#     "longitude": -6.248345,
#     "radius": 1000,
#     "categories": "food,busstations,visitorcenters", # search for dining and drinking, bus stops, and tourist information and service
#     "sort_by": "distance"
# }

# headers = {
#     "Accept": "application/json",
#     "Authorization": f"BEARER {yelp_api_key}"
# }

# yelp_result = requests.request("GET", url, params=params, headers=headers)

# print(yelp_result.json())

url = "https://api.yelp.com/v3/businesses/search"
params = {
    "latitude": 53.333653,
    "longitude": -6.248345,
    "radius": 1000,
    "categories": "bars,restaurants,coffeeshops,busstations,visitorcenters", # search for dining and drinking, bus stops, and tourist information and service
    "sort_by": "distance"
}

headers = {
    "Accept": "application/json",
    "Authorization": f"BEARER {yelp_api_key}"
}

yelp_result = requests.request("GET", url, params=params, headers=headers)

pprint.pprint(yelp_result.json())

{'businesses': [{'alias': 'canal-boat-restaurant-dublin',
                 'categories': [{'alias': 'irish', 'title': 'Irish'}],
                 'coordinates': {'latitude': 53.3329505,
                                 'longitude': -6.2488298},
                 'display_phone': '+353 1 473 1000',
                 'distance': 84.48742305707052,
                 'id': 'zCuDrfnrQSjvbqT95xjn5g',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/Tx7Ih26SUgXXZXuaAy2AEQ/o.jpg',
                 'is_closed': False,
                 'location': {'address1': 'Mespil Road',
                              'address2': '',
                              'address3': '',
                              'city': 'Dublin',
                              'country': 'IE',
                              'display_address': ['Mespil Road',
                                                  'Dublin 2',
                                                  'Republic of Ireland'],
                     

In [58]:
data = yelp_result.json().get('businesses') # extract data located in the 'businesses' header of the json file
yelp_df = pd.DataFrame(data) # convert location results to a pandas dataframe
yelp_df['latitude'] = latitude
yelp_df['longitude'] = longitude 
yelp_df['category'] = yelp_df['categories'].map(lambda x: x[0].get('title')) # extract only the first category name from a venue's category info
#yelp_df = yelp_df.drop('categories', axis = 1) # remove the 'categories' column which contains a list of dictionaries
yelp_df_1 = yelp_df.filter(['distance','name','review_count','rating','latitude','longitude','category'], axis=1) # return only columns of interest
yelp_df_1

,distance,name,review_count,rating,latitude,longitude,category
0,84.487423,Canal Boat Restaurant,2,2.5,53.351464,-6.255265,Irish
1,107.115303,La Peniche,13,3.0,53.351464,-6.255265,French
2,142.031898,Chopped,1,4.0,53.351464,-6.255265,Salad
3,144.987544,Boojum,2,3.0,53.351464,-6.255265,Mexican
4,169.648817,Keshk Cafe & Restaurant,49,4.5,53.351464,-6.255265,Mediterranean
5,183.582051,Beshoff Bros,30,3.5,53.351464,-6.255265,Fish & Chips
6,190.364757,Kanum,48,3.5,53.351464,-6.255265,Thai
7,196.017286,The Wellington,11,3.5,53.351464,-6.255265,Pubs
8,210.882500,Kerala Kitchen,30,4.5,53.351464,-6.255265,Indian
9,224.372058,Milano,35,3.5,53.351464,-6.255265,Italian


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Get the top 10 restaurants according to their rating

In [13]:
import pandas as pd
restaurants = pd.read_csv('../data/merged_clean_results.csv') # import merged results as a dataframe
restaurants = restaurants.drop(columns=restaurants.columns[0], axis=1) # remove duplicate index
restaurants_unique = restaurants.drop_duplicates(
    subset = ['name'],
    keep = 'first').reset_index(drop = True) # drop duplicate restaurants
restaurants_unique = restaurants_unique.loc[restaurants_unique['category'] != 'Tours'] # remove top result that was not a restaurant

restaurants_unique.sort_values(by=['rating_avg','review_count'], ascending=False).head(10) # sort dataframe by highest rating, then by highest number of reviews


,name,latitude,longitude,review_count,rating_avg,distance_avg,category
292,Green Bench Café,53.336021,-6.262980,101.0,5.0,86.864467,Salad
548,Fish Shop,53.347477,-6.285250,33.0,5.0,219.511391,Fish & Chips
762,Hacienda Bar,53.346874,-6.272976,24.0,5.0,238.464066,Pubs
837,Harkin's,53.341645,-6.297190,17.0,5.0,583.951089,Pubs
889,The Lord Mayor's Lounge,53.337824,-6.256035,11.0,5.0,112.243387,Lounges
272,DELISUZ,53.356307,-6.273717,8.0,5.0,392.901777,Breakfast & Brunch
1014,Forest & Marcy,53.332383,-6.252717,8.0,5.0,250.985016,Wine Bars
505,The Piper's Corner,53.348279,-6.254662,7.0,5.0,231.989691,Pubs
840,Groundstate,53.341645,-6.297190,6.0,5.0,612.849265,Coffee & Tea
852,Assassination Custard,53.337757,-6.267699,6.0,5.0,139.779619,Cafes


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The Yelp API had a larger number of results (almost double) than the Foursquare API, and included elements such as number of reviews.
The Foursquare API offered a wider variety of parameters that were not related to food and beverage services, but this would also mean that rating and review information would be less useful (since locations such as bus stops are unlikely to have ratings and reviews).

Both APIs had reasonably low levels of null values. Most null values occurred when merging the results from the two APIs.

Trial snippets

In [10]:
pprint.pprint(yelp_result.json())

{'businesses': [{'alias': 'fresh-dublin-4',
                 'categories': [{'alias': 'grocery', 'title': 'Grocery'},
                                {'alias': 'markets',
                                 'title': 'Fruits & Veggies'}],
                 'coordinates': {'latitude': 53.3492775,
                                 'longitude': -6.2785013},
                 'display_phone': '',
                 'distance': 132.309676155256,
                 'id': 'BOl7q5e_2JR8BmWMcIr3cQ',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/nNnYlGhuj7ZIiNOV35byfA/o.jpg',
                 'is_closed': False,
                 'location': {'address1': 'Market Square',
                              'address2': 'Smithfield',
                              'address3': None,
                              'city': 'Dublin',
                              'country': 'IE',
                              'display_address': ['Market Square',
                                                  '

Put your parsed results into a DataFrame